<a href="https://colab.research.google.com/github/heejinohn/lzho/blob/heejinohn/combine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install wrds datetime
import pandas as pd, numpy as np, wrds
import datetime as dt

# Read datasets

In [12]:
ticker.append?

In [14]:
ticker = pd.DataFrame()
for i in range(2019,2020,1):
    read = pd.read_parquet(f'/content/ticker_{i}.parquet.gzip')
    ticker = ticker.append(read)
col = ['date_utc','num_comments','score',
        'ticker_a','ticker_b']
ticker = ticker[col]

In [4]:
def comb_tic(x):
    if len(x[1]) == 0:
        return x[0]
    elif len(x[0]) == 0:
        return x[1]
    else:
       return list(set(x[0]).union(x[1]))

In [5]:
ticker['comb_tic'] = ticker[['ticker_a','ticker_b']].apply(comb_tic, axis=1)

In [10]:
ticker.shape

(71561, 6)

In [11]:
ticker[ticker['comb_tic'].apply(len) != 0]

,date_utc,num_comments,score,ticker_a,ticker_b,comb_tic
id,,,,,,
eiaz8e,2019-12-31 23:58:26+00:00,5,1,"[UBER, DD, LYFT]","[UBER, AMZN, LYFT]","[DD, UBER, AMZN, LYFT]"
eiatp9,2019-12-31 23:45:57+00:00,38,1,[DD],[WEAT],"[DD, WEAT]"
eiasq6,2019-12-31 23:43:38+00:00,0,1,[BSGM],[],[BSGM]
eial7r,2019-12-31 23:26:59+00:00,16,1,[TLRD],[PE],"[PE, TLRD]"
eia4wt,2019-12-31 22:50:25+00:00,0,1,"[PTI, LK, YOLO, XLE, NIO]",[],"[PTI, LK, YOLO, XLE, NIO]"
...,...,...,...,...,...,...
abej5o,2019-01-01 03:32:20+00:00,7,1,[GS],[],[GS]
abdmiy,2019-01-01 01:28:41+00:00,56,1,"[USO, DD]",[],"[USO, DD]"
abdgg0,2019-01-01 01:07:50+00:00,6,1,[SPX],[],[SPX]
